In [1]:
# import Libraries

import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from secret_key import my_pinecone_key

# Securely set up environment variables for API keys (This should be done outside of your script, in your environment)
os.environ['OPENAI_API_KEY'] = my_pinecone_key

In [5]:
## Lets Read the document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [6]:
#!pip install pypdf

In [7]:
!pwd

/Users/ritikaverma/Desktop/internship/Athina


In [8]:
doc=read_doc('Doc/')
len(doc)

44

In [9]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=100,chunk_overlap=10):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [10]:
documents=chunk_data(docs=doc)
len(documents)

44

In [11]:
## Embedding Technique Of OPENAI
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x112425290>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x1124350d0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [12]:
vectors=embeddings.embed_query("How are you?")
len(vectors)

1536

In [13]:
!pip install --upgrade pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 3.8 MB/s eta 0:00:00a 0:00:01


In [14]:
import os
from pinecone import Pinecone, ServerlessSpec
    
pc = Pinecone(api_key = my_pinecone_key)
    
    
    
    

# Now do stuff
if 'athina' not in pc.list_indexes().names():
    pc.create_index(
        name='athina', 
        dimension=1536, 
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [15]:
#!pip install -U langchain-pinecone

In [17]:
#from langchain_pinecone import PineconeVectorStore

#index_name = "lang"

#index = PineconeVectorStore.from_documents(doc, embeddings, index_name=index_name)

In [18]:
from langchain_pinecone import Pinecone

index=Pinecone.from_documents(doc,embeddings,index_name='athina')

In [1]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=5):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [2]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [21]:
llm=OpenAI(model_name="gpt-3.5-turbo-instruct",temperature=0)
chain=load_qa_chain(llm,chain_type="stuff")

/Users/ritikaverma/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [22]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [36]:
our_query = "What does the document talk about?"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content='Page 41Everything else\nEverything else\nOur contract with you\nWe aim to always be fair and reasonable and to \nact quickly whenever you need to make a claim under this policy. If you feel we haven’t met this, \nwe’ll try to do everything possible to deal with your complaint quickly and fairly.\nThis policy is evidence of the contract between \nyou and us, U K Insurance Limited, and is based \non information you’ve given to us.\nIn return for receiving and accepting the \npremium, we’ll provide insurance under this policy for the sections shown on your car \ninsurance details. \nThe laws that apply to this contract You and we \nmay choose which law will apply to this policy. \nUnless both parties agree otherwise, English law will apply. However, if you are resident in Jersey, Guernsey, Alderney or the Isle of Man, the law of the island where you are resident will always apply to your policy and any dispute in relation to it will be within the jurisdiction of th

In [37]:
our_query = "Please summarize this documents and do it in bullets."
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content='Page 2FAQs  3\nGlossary  4\nMaking a claim  6\nWhat your cover includes  8\nSection 1: Liability  11\nSection 2: Fire and theft  14\nSection 3: Courtesy car  17\nSection 4: Accidental damage  18\nSection 5: Windscreen damage  20\nSection 6: Personal benefits  21\nSection 7: Motor Legal Cover  23\nSection 8: Guaranteed Hire Car Plus  28\nSection 9: Protected No Claim Discount  30Where you can drive  31\nLosses we don’t cover  33\nOther conditions you need to know about  36\nHow the policy works  37\nEverything else  41\nIf you have a complaint  42\nIf you’re in an accident  43\nHow to get in touch  Back coverContentsWelcome to Churchill\nThis booklet tells you about your car insurance\nAbout the policy\nThe policy is made up of:\n >This booklet.\n >Your car insurance details.\n >Your certificate (or certificates)  \nof motor insurance.\nIf the policy includes Green Flag breakdown cover:\n >Your breakdown cover and your car \ninsurance are part of the same policy.

In [23]:
our_query = "What is the policy on fire damage?"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content='Page 14You’re not covered under this \nsub-section for\n8We won’t cover any loss, damage or injury:\n• That takes place outside of Great Britain. • Caused by your automated car driving \nitself at any time or place that the use of \nautomated functions is unlawful. \n• To the extent that an accident was caused \nor contributed to by any party suffering loss, damage or injury. \n• To an insured person if the accident is \ncaused by a failure to install safety critical updates to your automated car or its \nsoftware has been altered without the approval of the manufacturer. We may also require an insured person to repay us \nany amounts that we are required by law to pay. \n• To the person in charge of your automated \ncar where the accident was wholly due to that person’s negligence in allowing your automated car to begin driving itself when it was not appropriate to do so. \n• To property which an insured person owns \nor is responsible for.\n• That’s covered by

/Users/ritikaverma/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 The policy covers damage to your car caused by fire, lightning, or an explosion. The insurance company can choose to repair, replace, or repay for the damage.


In [24]:
our_query = "Give me further details on policy for fire damage?"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content='Your car insurance \npolicy booklet\n', metadata={'page': 0.0, 'source': 'Doc/policy-booklet-0923.pdf'}), Document(page_content='Your car insurance \npolicy booklet\n', metadata={'page': 0.0, 'source': 'Doc/policy-booklet-0923.pdf'})]
 I don't know, as I don't have access to the specific policy booklet. It would be best to refer to the booklet for further details on coverage for fire damage.
